**Data Science Section 002 Group 7 Project Proposal**

**Predicting the Presence or Absence of Heart Disease based on Specific Variables.**

Predictive data analysis plays an important role in the field of medical diagnostics. As cardiovascular disease continues to be a leading cause of global mortality, the development of efficient predictive models becomes crucial for detection and intervention. In this analysis, we will be predicting the presence or absence of heart disease in patients using the key parameters of age, cholesterol levels, and resting blood pressure. For this analysis, we will be using the Heart Disease dataset from the UCI Machine Learning Repository, and focusing on the Cleveland database. The original database contains 76 attributes, but a version containing a subset of only 14 of those attributes will be used for this analysis. Out of those 14 variables we are only choosing 5 to observe in our project. We have chosen age, trestbps (resting blood pressure), chol (serum cholesterol in mg/dl), thalach (maximum heart rate achieved) and num (the diagnosis of heart disease) which we will rename to "diagnosis". We chose these specific variables because we think that they would most significantly impact the presence of heart disease in a patient.

By the end of this analysis, we expect to gain insight into the complex relationships between our chosen variables and the presence of heart disease in patients. Such findings would contribute to medical research regarding cardiovascular disease by providing insight into trends in the broader population of where the data was collected. Furthermore, this insight would allow healthcare systems to identify and focus on individuals at higher risk of heart diseases, and enable earlier intervention, improving patient outcomes. This insight could also contribute to reduced healthcare costs, as early detection and intervention could reduce the need for costly medical procedures associated with later stages of heart disease. A variety of questions could arise from the results of this analysis as well, such as whether the accuracy of the model could be improved using other information. For example, one could inquire about how lifestyle and behavioral factors such as smoking habits, exercise and diet interact with cholesterol, age, and blood pressure to influence the risk of heart disease in patients. We could also question whether the model and its findings generalize to different populations, and discuss how to make the model applicable and effective for more diverse populations. 



To further understand our data, some preliminary exploratory data analysis is conducted below. To gain access to useful functions for reading and analysing our data, the following libraries must be installed:

In [14]:
library(tidyverse)
library(tidymodels)

We can now read our dataset into R and assign it to an object called cleveland_data. We are also going to limit the amount of rows shown when previewing the dataset:

In [15]:
#Demostration that the dataset can be read from the web into R:
options(repr.matrix.max.rows = 10)

url <- "https://raw.githubusercontent.com/Nitro-87/Group-2-Project-Proposal/main/processed.cleveland.data"
cleveland_data <- read_delim(url, delim= "," , col_names=FALSE)

cleveland_data

Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): X12, X13
dbl (12): X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X14

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
45,1,1,110,264,0,0,132,0,1.2,2,0.0,7.0,1
68,1,4,144,193,1,0,141,0,3.4,2,2.0,7.0,2
57,1,4,130,131,0,0,115,1,1.2,2,1.0,7.0,3


Since our data contains no column names, we rename each columns with the appropriate variable name below. We also replace the numerial values under our categorical variables with what they are supposed to represent, to better understand what each variable reveals to us.

In [16]:
#Renaming columns:

cleveland_data <- rename(cleveland_data,
       age = X1,
       sex = X2,
       chest_pain_type = X3,
       trestbps = X4,
       chol = X5,
       fbs = X6,    
       restecg = X7,
       thalach = X8,
       exang = X9,
       oldpeak = X10,
       slope = X11,
       ca = X12,
       thal = X13,
       diagnosis = X14)

#skipping rows with missing values
cleveland_data<- cleveland_data |> drop_na()

glimpse(cleveland_data)

Rows: 303
Columns: 14
$ age             <dbl> 63, 67, 67, 37, 41, 56, 62, 57, 63, 53, 57, 56, 56, 44…
$ sex             <dbl> 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, …
$ chest_pain_type <dbl> 1, 4, 4, 3, 2, 2, 4, 4, 4, 4, 4, 2, 3, 2, 3, 3, 2, 4, …
$ trestbps        <dbl> 145, 160, 120, 130, 130, 120, 140, 120, 130, 140, 140,…
$ chol            <dbl> 233, 286, 229, 250, 204, 236, 268, 354, 254, 203, 192,…
$ fbs             <dbl> 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, …
$ restecg         <dbl> 2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 0, 0, 0, 0, …
$ thalach         <dbl> 150, 108, 129, 187, 172, 178, 160, 163, 147, 155, 148,…
$ exang           <dbl> 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, …
$ oldpeak         <dbl> 2.3, 1.5, 2.6, 3.5, 1.4, 0.8, 3.6, 0.6, 1.4, 3.1, 0.4,…
$ slope           <dbl> 3, 2, 2, 3, 1, 1, 3, 1, 2, 3, 2, 2, 2, 1, 1, 1, 3, 1, …
$ ca              <chr> "0.0", "3.0", "2.0", "0.0", "0.0", "0.0", "2.0", "0.0"…
$ thal            

In [17]:
#renaming categorical variables:

#renaming sex values
cleveland_data <- cleveland_data |>
mutate(sex = as_factor(sex))|>
mutate(sex = fct_recode(sex, "female" = "0", 
                            "male" = "1"))
#renaming chest pain values
cleveland_data <- cleveland_data |>
mutate(chest_pain_type = as_factor(chest_pain_type))|>
mutate(chest_pain_type = fct_recode(chest_pain_type, 
                            "typical angina" = "1", 
                            "atypical angina" = "2", 
                            "non-anginal_pain" = "3", 
                            "asymptomatic" = "4"))
#renaming fbs values
cleveland_data <- cleveland_data |>
mutate(fbs = as_factor(fbs))|>
mutate(fbs = fct_recode(fbs, "false" = "0", 
                            "true" = "1"))

#renaming exang values
cleveland_data <- cleveland_data |>
mutate(exang = as_factor(exang))|>
mutate(exang = fct_recode(exang, "no" = "0", 
                            "yes" = "1"))
#renaming slope values
cleveland_data <- cleveland_data |>
mutate(slope = as_factor(slope))|>
mutate(slope = fct_recode(thal,"upsloping" = "1", 
                            "flat" = "2", 
                            "downsloping" = "3"))
#renaming thal values
cleveland_data <- cleveland_data |>
mutate(thal = as_factor(thal))|>
mutate(thal = fct_recode(thal, "reversible defect" = "7.0", 
                            "fixed defect" = "6.0", 
                            "normal" = "3.0"))
#renaming diagnosis values
cleveland_data <- cleveland_data |>
mutate(diagnosis = as_factor(diagnosis))|>
mutate(diagnosis = fct_recode(diagnosis, 
                            "absent" = "0",
                              "present" = "1", 
                            "present" = "2", 
                            "present" = "3", 
                            "present" = "4"))

glimpse(cleveland_data)

Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `slope = fct_recode(thal, upsloping = "1", flat = "2",
  downsloping = "3")`.
Caused by warning:
! Unknown levels in `f`: 1, 2, 3”


Rows: 303
Columns: 14
$ age             <dbl> 63, 67, 67, 37, 41, 56, 62, 57, 63, 53, 57, 56, 56, 44…
$ sex             <fct> male, male, male, male, female, male, female, female, …
$ chest_pain_type <fct> typical angina, asymptomatic, asymptomatic, non-angina…
$ trestbps        <dbl> 145, 160, 120, 130, 130, 120, 140, 120, 130, 140, 140,…
$ chol            <dbl> 233, 286, 229, 250, 204, 236, 268, 354, 254, 203, 192,…
$ fbs             <fct> true, false, false, false, false, false, false, false,…
$ restecg         <dbl> 2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 0, 0, 0, 0, …
$ thalach         <dbl> 150, 108, 129, 187, 172, 178, 160, 163, 147, 155, 148,…
$ exang           <fct> no, yes, yes, no, no, no, no, yes, no, yes, no, no, ye…
$ oldpeak         <dbl> 2.3, 1.5, 2.6, 3.5, 1.4, 0.8, 3.6, 0.6, 1.4, 3.1, 0.4,…
$ slope           <fct> 6.0, 3.0, 7.0, 3.0, 3.0, 3.0, 3.0, 3.0, 7.0, 7.0, 6.0,…
$ ca              <chr> "0.0", "3.0", "2.0", "0.0", "0.0", "0.0", "2.0", "0.0"…
$ thal            

We then want to filter our dataset to only include the diagnosis and variables that we want to observe which are age, chol, tresbps and thalach.

In [18]:
cleveland_data_select <- select(cleveland_data, age, trestbps, chol, thalach, diagnosis)
cleveland_data_select

age,trestbps,chol,thalach,diagnosis
<dbl>,<dbl>,<dbl>,<dbl>,<fct>
63,145,233,150,absent
67,160,286,108,present
67,120,229,129,present
37,130,250,187,absent
41,130,204,172,absent
⋮,⋮,⋮,⋮,⋮
45,110,264,132,present
68,144,193,141,present
57,130,131,115,present


Now that our data is tidy and filtered, we can split it into a training set to construct our model, and a testing set to test our model's effectiveness.

In [19]:
#splitting data into training and testing sets

cleveland_data_split <- initial_split(cleveland_data_select, prop = 0.75, strata = diagnosis)
training_set <- training(cleveland_data_split)
testing_set <- testing(cleveland_data_split)

training_set
testing_set

age,trestbps,chol,thalach,diagnosis
<dbl>,<dbl>,<dbl>,<dbl>,<fct>
37,130,250,187,absent
41,130,204,172,absent
56,120,236,178,absent
57,120,354,163,absent
57,140,192,148,absent
⋮,⋮,⋮,⋮,⋮
59,164,176,90,present
45,110,264,132,present
68,144,193,141,present


age,trestbps,chol,thalach,diagnosis
<dbl>,<dbl>,<dbl>,<dbl>,<fct>
63,145,233,150,absent
62,140,268,160,present
53,140,203,155,present
52,172,199,162,absent
48,110,229,168,present
⋮,⋮,⋮,⋮,⋮
59,134,204,162,present
66,146,278,152,absent
47,130,253,179,absent


Below is a table conducting exploratory analysis on our training dataset. This table tells us the number and percentage of observations in each class group (whether there is a presence/absence of heart disease).

In [20]:
num_obs <- nrow(training_set)
training_set |>
  group_by(diagnosis) |>
  summarize(
    count = n(),
    percentage = n() / num_obs * 100)

diagnosis,count,percentage
<fct>,<int>,<dbl>
absent,123,54.18502
present,104,45.81498
